# This will be feature selection based on the paper by kniep HC in which they get tid off all those tumors which volume is less than 125 and select the 10 lasrgest metastsis per patient. I am going to go with 15 for now. The rest would be applying the same codes as before. Only the radiomic features of the 3 classes: Melanomma, Lung, Breast (0, 1, 2).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate, KFold, RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, make_scorer
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import chi2
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import VotingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_selection import VarianceThreshold
from collections import Counter
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.pipeline import Pipeline as imbpipeline
from mrmr import mrmr_classif
from scipy import stats
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_selection import f_classif

In [2]:
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, multi_class='ovr', average=average)

from sklearn.metrics import roc_auc_score

def class_vs_rest_roc_auc_score(y_test, y_pred):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_bin = lb.transform(y_test)
    y_pred_bin = lb.transform(y_pred)
    roc_auc_scores = {}

    for i, class_label in enumerate(lb.classes_):
        class_indices = lb.transform([class_label])[0]
        rest_indices = ~class_indices
        y_test_class = y_test_bin[:, i]
        y_test_rest = y_test_bin[:, rest_indices]
        y_pred_class = y_pred_bin[:, i]
        y_pred_rest = y_pred_bin[:, rest_indices]
        roc_auc_scores[class_label] = roc_auc_score(y_test_class, y_pred_class, multi_class='ovr', average="macro")

    return roc_auc_scores

def calculate_specificity(cm):
    num_classes = cm.shape[0]
    specificity = {}

    for i in range(num_classes):
        tp = cm[i, i]
        tn = np.sum(np.delete(np.delete(cm, i, axis=0), i, axis=1))
        fp = np.sum(cm[:, i]) - tp
        fn = np.sum(cm[i, :]) - tp

        specificity[i] = 100*(tn / (tn + fp))

    return specificity

def calculate_sensitivity(cm):
    num_classes = cm.shape[0]
    sensitivity = {}

    for i in range(num_classes):
        tp = cm[i, i]
        fn = np.sum(cm[i, :]) - tp

        sensitivity[i] = 100*(tp / (tp + fn))

    return sensitivity

In [3]:
df = pd.read_excel('all_patients.xlsx')
#df = df.set_index(['subjid'])
df = df.drop('number_annotations', axis = 1)
df = df[df.shape_VoxelVolume > 125]
df.drop(df[df['labels'] == 3].index, inplace = True) ## dropping label of the call OTHERS
df = df.set_index(['subjid', 'label'])
confidence = 0.95
seed = 0
df

shape_Elongation  shape_Flatness  shape_LeastAxisLength   
subjid   label                                                            
Oslo01   1              0.829356        0.682981               8.512717  \
         2              0.944143        0.900681              14.133927   
Oslo02   4              0.849295        0.841082               8.703783   
         5              0.939828        0.821847              12.093804   
Oslo03   1              0.930698        0.629604              10.914986   
...                          ...             ...                    ...   
Stan_328 1              0.810571        0.325076               8.769420   
         8              0.967496        0.597282               6.131668   
         11             0.561746        0.506390               5.004740   
         12             0.959775        0.756903               5.164755   
Stan_338 4              0.819799        0.618051               8.548537   

                shape_MajorAxisLength  shape_Maximum2DDiameterColumn   
subjid   label                                                         
Oslo01   1                  12.464063                      14.866069  \
         2                  15.692487                      17.804494   
Oslo02   4                  10.348315                      10.816654   
         5                  14.715396                      17.262677   
Oslo03   1                  17.336268                      20.615528   
...                               ...                            ...   
Stan_328 1                  26.976501                      31.384710   
         8                  10.265944                      11.661904   
         11                  9.883181                       8.246211   
         12                  6.823538                       7.615773   
Stan_338 4                  13.831433                      14.142136   

                shape_Maximum2DDiameterRow  shape_Maximum2DDiameterSlice   
subjid   label                                                             
Oslo01   1                       14.560220                     12.529964  \
         2                       19.416488                     18.681542   
Oslo02   4                       11.704700                     11.704700   
         5                       17.888544                     17.492856   
Oslo03   1                       18.439089                     19.416488   
...                                    ...                           ...   
Stan_328 1                       28.160256                     27.459060   
         8                       12.369317                     12.165525   
         11                      11.704700                     10.000000   
         12                       8.602325                      8.602325   
Stan_338 4                       15.297059                     17.204651   

                shape_Maximum3DDiameter  shape_MeshVolume   
subjid   label                                              
Oslo01   1                    15.394804        728.083333  \
         2                    19.467922       2320.625000   
Oslo02   4                    12.449900        538.666667   
         5                    18.000000       1681.250000   
Oslo03   1                    20.712315       2068.958333   
...                                 ...               ...   
Stan_328 1                    33.075671       3310.541667   
         8                    12.688578        375.166667   
         11                   12.124356        159.791667   
         12                    8.602325        149.333333   
Stan_338 4                    17.291616        821.541667   

                shape_MinorAxisLength  ...  glszm_ZoneVariance   
subjid   label                         ...                       
Oslo01   1                  10.337151  ...        3.039725e+04  \
         2                  14.815951  ...        1.935116e+05   
Oslo02   4                   8.788767  ...        2.8454

In [4]:
# we select X and y
X = df.iloc[:, :-1] #leaving the sex and age features out for now
y = df.iloc[:, -1:]
print(Counter(y.values.ravel()))

Counter({1: 321, 2: 76, 0: 41})


There is an obvious inbalane of the labes wih a great majority coming from the lung. To counter attck this inbalance we eill make use of oversampling and undersapmling techniques that have proven to be useful in literature

## pipelines

In [7]:
pipelines = {
    'rf': imbpipeline([('smoteen', SMOTEENN(random_state=0)),
                       ('clf', RandomForestClassifier(random_state=0, class_weight="balanced"))]),
    'svc': imbpipeline([('smoteen', SMOTEENN(random_state=0)),
                        ('clf', CalibratedClassifierCV(LinearSVC(multi_class='ovr', random_state=0, class_weight="balanced")))])
    }

param_grids = {
    'rf': {'clf__n_estimators': [100, 250, 1000, 3500]},
    'svc': {'clf__base_estimator__C': [0.1, 1, 10], 'clf__base_estimator__max_iter': [10000, 20000]}
}

cv_outer = RepeatedStratifiedKFold(n_splits=6, n_repeats=6, random_state=seed)

## Age

In [163]:
X_Age = X['Age']

# Store the scores and metrics
scores_rf = {'acc': [], 'SD_acc': [], 'CI_acc': [], 'auc_tot': [], 'SD_auc_tot': [], 'CI_auc_tot': [], 'auc_class': [], 'SD_auc_class': [], 'CI_auc_class': [], 'Sen_mean': [], 'Sen_CI': [], 'Sen_SD': [], 'Spe_mean': [], 'Spe_CI': [], 'Spe_sd': []}
scores_svc = {'acc': [], 'SD_acc': [], 'CI_acc': [], 'auc_tot': [], 'SD_auc_tot': [], 'CI_auc_tot': [], 'auc_class': [], 'SD_auc_class': [], 'CI_auc_class': [], 'Sen_mean': [], 'Sen_CI':[], 'Sen_SD': [], 'Spe_mean': [], 'Spe_CI': [], 'Spe_sd': []}
best_features_RF = []

for name, pipeline in pipelines.items():
    print(name)
    # Initialize lists to store scores for each fold
    fold_acc = []
    fold_auc_tot = []
    fold_auc_class = []
    sensi = []
    speci = []
    features = []
    p_value = []
    feat = []
    value = []
    for train_index, test_index in cv_outer.split(X_Age, y):
        X_train, X_test = X_Age.iloc[train_index], X_Age.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        #scaler = StandardScaler()
        #X_train_new = X_train.values.reshape(-1, 1)
        #X_test_new = X_test.values.reshape(-1, 1)

        scaler = StandardScaler()
        X_train_new = scaler.fit_transform(X_train.values.reshape(-1, 1))
        X_test_new = scaler.transform(X_test.values.reshape(-1, 1))

        #X_train_new = pd.DataFrame(X_train_normed, columns=X_train.columns, index=X_train.index)
        #X_test_new = pd.DataFrame(X_test_normed, columns=X_test.columns, index=X_test.index)

        #cv_inner = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=seed)
        grid_search = GridSearchCV(pipeline, param_grids[name], cv=5, scoring='roc_auc_ovr', error_score="raise")
        grid_search.fit(X_train_new, y_train)
        print(f'{name}', grid_search.best_score_)
        clf = grid_search.best_estimator_
        print(clf)
        clf.fit(X_train_new, y_train)
        y_pred = clf.predict(X_test_new)
        accuracy = accuracy_score(y_test, y_pred)
        print("Accuracy:", f'{name}', accuracy)
        num_classes = len(np.unique(y_test))
        cm = confusion_matrix(y_test, y_pred)
        print(cm)
        specificity = calculate_specificity(cm)
        sensitivity = calculate_sensitivity(cm)


        accuracy = accuracy_score(y_test, y_pred)
        fold_acc.append(accuracy)

        # Calculate AUC score
        auc_tot = multiclass_roc_auc_score(y_test, y_pred)
        fold_auc_tot.append(auc_tot)

        auc_class = class_vs_rest_roc_auc_score(y_test, y_pred)
        fold_auc_class.append(auc_class)

        speci.append(specificity)
        sensi.append(sensitivity)

        print('Auc Score', multiclass_roc_auc_score(y_test, y_pred))
        print('Auc Score OvR', class_vs_rest_roc_auc_score(y_test, y_pred))

    # Calculate average scores across folds
    if name == 'rf':
        #acc
        avg_acc = np.mean(fold_acc)
        scores_rf['acc'].append(avg_acc)
        scores_rf['acc'] = scores_rf['acc'][-1]
        ci = stats.t.interval(confidence, len(fold_acc) - 1, loc=np.mean(fold_acc), scale=stats.sem(fold_acc))
        scores_rf['CI_acc'].append(ci)
        scores_rf['SD_acc'].append(np.std(fold_acc))

        #auc_tot
        avg_auc_tot = np.mean(fold_auc_tot)
        scores_rf['auc_tot'].append(avg_auc_tot)
        scores_rf['auc_tot'] = scores_rf['auc_tot'][-1]
        ci = stats.t.interval(confidence, len(fold_auc_tot) - 1, loc=np.mean(fold_auc_tot), scale=stats.sem(fold_auc_tot))
        scores_rf['CI_auc_tot'].append(ci)
        scores_rf['SD_auc_tot'].append(np.std(fold_auc_tot))

        #auc_class
        list_0 = list(d[0] for d in fold_auc_class)
        list_1 = list(d[1] for d in fold_auc_class)
        list_2 = list(d[2] for d in fold_auc_class)

        avg_auc_class = {0: np.mean(list_0), 1: np.mean(list_1), 2: np.mean(list_2)}
        scores_rf['auc_class'].append(avg_auc_class)

        sd_auc_class = {0: np.std(list_0), 1: np.std(list_1), 2: np.std(list_2)}
        scores_rf['SD_auc_class'].append(sd_auc_class)

        ci_0 = stats.t.interval(confidence, len(list_0) - 1, loc=np.mean(list_0), scale=stats.sem(list_0))
        ci_1 = stats.t.interval(confidence, len(list_1) - 1, loc=np.mean(list_1), scale=stats.sem(list_1))
        ci_2 = stats.t.interval(confidence, len(list_2) - 1, loc=np.mean(list_2), scale=stats.sem(list_2))

        CI_auc_class = {0: ci_0, 1: ci_1, 2: ci_2}
        scores_rf['CI_auc_class'].append(CI_auc_class)

        #sensitivity
        list_0 = list(d[0] for d in sensi)
        list_1 = list(d[1] for d in sensi)
        list_2 = list(d[2] for d in sensi)

        avg_sensi_class = {0: np.mean(list_0), 1: np.mean(list_1), 2: np.mean(list_2)}
        scores_rf['Sen_mean'].append(avg_sensi_class)

        sd_sensi_class = {0: np.std(list_0), 1: np.std(list_1), 2: np.std(list_2)}
        scores_rf['Sen_SD'].append(sd_sensi_class)

        ci_0 = stats.t.interval(confidence, len(list_0) - 1, loc=np.mean(list_0), scale=stats.sem(list_0))
        ci_1 = stats.t.interval(confidence, len(list_1) - 1, loc=np.mean(list_1), scale=stats.sem(list_1))
        ci_2 = stats.t.interval(confidence, len(list_2) - 1, loc=np.mean(list_2), scale=stats.sem(list_2))

        CI_sen_class = {0: ci_0, 1: ci_1, 2: ci_2}
        scores_rf['Sen_CI'].append(CI_sen_class)

        #specificity
        list_0 = list(d[0] for d in speci)
        list_1 = list(d[1] for d in speci)
        list_2 = list(d[2] for d in speci)

        avg_speci_class = {0: np.mean(list_0), 1: np.mean(list_1), 2: np.mean(list_2)}
        scores_rf['Spe_mean'].append(avg_speci_class)
        sd_speci_class = {0: np.std(list_0), 1: np.std(list_1), 2: np.std(list_2)}
        scores_rf['Spe_sd'].append(sd_speci_class)

        ci_0 = stats.t.interval(confidence, len(list_0) - 1, loc=np.mean(list_0), scale=stats.sem(list_0))
        ci_1 = stats.t.interval(confidence, len(list_1) - 1, loc=np.mean(list_1), scale=stats.sem(list_1))
        ci_2 = stats.t.interval(confidence, len(list_2) - 1, loc=np.mean(list_2), scale=stats.sem(list_2))

        CI_spe_class = {0: ci_0, 1: ci_1, 2: ci_2}
        scores_rf['Spe_CI'].append(CI_spe_class)

        continue
    else:
        # acc
        avg_acc = np.mean(fold_acc)
        scores_svc['acc'].append(avg_acc)
        scores_svc['acc'] = scores_svc['acc'][-1]
        ci = stats.t.interval(confidence, len(fold_acc) - 1, loc=np.mean(fold_acc), scale=stats.sem(fold_acc))
        scores_svc['CI_acc'].append(ci)
        scores_svc['SD_acc'].append(np.std(fold_acc))

        # auc_tot
        avg_auc_tot = np.mean(fold_auc_tot)
        scores_svc['auc_tot'].append(avg_auc_tot)
        scores_svc['auc_tot'] = scores_svc['auc_tot'][-1]
        ci = stats.t.interval(confidence, len(fold_auc_tot) - 1, loc=np.mean(fold_auc_tot), scale=stats.sem(fold_auc_tot))
        scores_svc['CI_auc_tot'].append(ci)
        scores_svc['SD_auc_tot'].append(np.std(fold_auc_tot))

        # auc_class
        list_0 = list(d[0] for d in fold_auc_class)
        list_1 = list(d[1] for d in fold_auc_class)
        list_2 = list(d[2] for d in fold_auc_class)

        avg_auc_class = {0: np.mean(list_0), 1: np.mean(list_1), 2: np.mean(list_2)}
        scores_svc['auc_class'].append(avg_auc_class)

        sd_auc_class = {0: np.std(list_0), 1: np.std(list_1), 2: np.std(list_2)}
        scores_svc['SD_auc_class'].append(sd_auc_class)

        ci_0 = stats.t.interval(confidence, len(list_0) - 1, loc=np.mean(list_0), scale=stats.sem(list_0))
        ci_1 = stats.t.interval(confidence, len(list_1) - 1, loc=np.mean(list_1), scale=stats.sem(list_1))
        ci_2 = stats.t.interval(confidence, len(list_2) - 1, loc=np.mean(list_2), scale=stats.sem(list_2))

        CI_auc_class = {0: ci_0, 1: ci_1, 2: ci_2}
        scores_svc['CI_auc_class'].append(CI_auc_class)

        # sensitivity
        list_0 = list(d[0] for d in sensi)
        list_1 = list(d[1] for d in sensi)
        list_2 = list(d[2] for d in sensi)

        avg_sensi_class = {0: np.mean(list_0), 1: np.mean(list_1), 2: np.mean(list_2)}
        scores_svc['Sen_mean'].append(avg_sensi_class)

        sd_sensi_class = {0: np.std(list_0), 1: np.std(list_1), 2: np.std(list_2)}
        scores_svc['Sen_SD'].append(sd_sensi_class)

        ci_0 = stats.t.interval(confidence, len(list_0) - 1, loc=np.mean(list_0), scale=stats.sem(list_0))
        ci_1 = stats.t.interval(confidence, len(list_1) - 1, loc=np.mean(list_1), scale=stats.sem(list_1))
        ci_2 = stats.t.interval(confidence, len(list_2) - 1, loc=np.mean(list_2), scale=stats.sem(list_2))

        CI_sen_class = {0: ci_0, 1: ci_1, 2: ci_2}
        scores_svc['Sen_CI'].append(CI_sen_class)

        # specificity
        list_0 = list(d[0] for d in speci)
        list_1 = list(d[1] for d in speci)
        list_2 = list(d[2] for d in speci)

        avg_speci_class = {0: np.mean(list_0), 1: np.mean(list_1), 2: np.mean(list_2)}
        scores_svc['Spe_mean'].append(avg_speci_class)

        sd_speci_class = {0: np.std(list_0), 1: np.std(list_1), 2: np.std(list_2)}
        scores_svc['Spe_sd'].append(sd_speci_class)

        ci_0 = stats.t.interval(confidence, len(list_0) - 1, loc=np.mean(list_0), scale=stats.sem(list_0))
        ci_1 = stats.t.interval(confidence, len(list_1) - 1, loc=np.mean(list_1), scale=stats.sem(list_1))
        ci_2 = stats.t.interval(confidence, len(list_2) - 1, loc=np.mean(list_2), scale=stats.sem(list_2))

        CI_spe_class = {0: ci_0, 1: ci_1, 2: ci_2}
        scores_svc['Spe_CI'].append(CI_spe_class)

rf
rf 0.5504537484802471
Pipeline(steps=[('smoteen', SMOTEENN(random_state=0)),
                ('clf',
                 RandomForestClassifier(class_weight='balanced',
                                        random_state=0))])
Accuracy: rf 0.589041095890411
[[ 5  2  0]
 [15 35  4]
 [ 3  6  3]]
Auc Score 0.6421800312726399
Auc Score OvR {0: 0.7207792207792209, 1: 0.6135477582846004, 2: 0.5922131147540984}
rf 0.5367136897133904
Pipeline(steps=[('smoteen', SMOTEENN(random_state=0)),
                ('clf',
                 RandomForestClassifier(class_weight='balanced',
                                        n_estimators=1000, random_state=0))])
Accuracy: rf 0.6027397260273972
[[ 2  4  1]
 [12 35  7]
 [ 3  2  7]]
Auc Score 0.6404976708758706
Auc Score OvR {0: 0.5292207792207791, 1: 0.6661793372319689, 2: 0.7260928961748635}
rf 0.5089627789786855
Pipeline(steps=[('smoteen', SMOTEENN(random_state=0)),
                ('clf',
                 RandomForestClassifier(class_weight='balanced',

In [164]:
for name, score in scores_svc.items():
    print('svc', f'{name}: {score}')

for name, score in scores_rf.items():
    print('rf', f'{name}: {score}')

svc acc: 0.6282343987823441
svc SD_acc: [0.13576905061675495]
svc CI_acc: [(0.5816451308594414, 0.6748236667052468)]
svc auc_tot: 0.49500489742853304
svc SD_auc_tot: [0.13576905061675495]
svc CI_auc_tot: [(0.48461070736254414, 0.505399087494522)]
svc auc_class: [{0: 0.49283973351137533, 1: 0.4990875854614407, 2: 0.49308737331278313}]
svc SD_auc_class: [{0: 0.029538748273660022, 1: 0.04989268331782548, 2: 0.034743373510620165}]
svc CI_auc_class: [{0: (0.4827034863241324, 0.5029759806986183), 1: (0.48196686774317266, 0.5162083031797088), 2: (0.4811651544442068, 0.5050095921813594)}]
svc Sen_mean: [{0: 5.621693121693122, 1: 82.80728317415948, 2: 9.081196581196583}]
svc Sen_CI: [{0: (0.533368653072503, 10.71001759031374), 1: (75.59409771402876, 90.0204686342902), 2: (5.428092787998811, 12.734300374394355)}]
svc Sen_SD: [{0: 14.82824292233659, 1: 21.020449247349145, 2: 10.64576577223079}]
svc Spe_mean: [{0: 92.94625358058194, 1: 17.01023391812865, 2: 89.53627808136004}]
svc Spe_CI: [{0: (87

## Sex and Age

In [116]:
X_sex_age = X[['Age', 'F', 'M']]

# Store the scores and metrics
scores_rf = {'acc': [], 'SD_acc': [], 'CI_acc': [], 'auc_tot': [], 'SD_auc_tot': [], 'CI_auc_tot': [], 'auc_class': [], 'SD_auc_class': [], 'CI_auc_class': [], 'Sen_mean': [], 'Sen_CI': [], 'Sen_SD': [], 'Spe_mean': [], 'Spe_CI': [], 'Spe_sd': []}
scores_svc = {'acc': [], 'SD_acc': [], 'CI_acc': [], 'auc_tot': [], 'SD_auc_tot': [], 'CI_auc_tot': [], 'auc_class': [], 'SD_auc_class': [], 'CI_auc_class': [], 'Sen_mean': [], 'Sen_CI':[], 'Sen_SD': [], 'Spe_mean': [], 'Spe_CI': [], 'Spe_sd': []}
best_features_RF = []

for name, pipeline in pipelines.items():
    # Initialize lists to store scores for each fold
    fold_acc = []
    fold_auc_tot = []
    fold_auc_class = []
    sensi = []
    speci = []
    features = []
    p_value = []
    feat = []
    value = []
    for train_index, test_index in cv_outer.split(X_sex_age, y):
        X_train_new, X_test_new = X_sex_age.iloc[train_index], X_sex_age.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        cv_inner = RepeatedStratifiedKFold(n_splits=4, n_repeats=4, random_state=seed)
        grid_search = GridSearchCV(pipeline, param_grids[name], cv=cv_inner, scoring='roc_auc_ovr', error_score="raise")
        grid_search.fit(X_train_new, y_train)
        clf = grid_search.best_estimator_
        print(clf)
        clf.fit(X_train_new, y_train)
        y_pred = clf.predict(X_test_new)
        accuracy = accuracy_score(y_test, y_pred)
        num_classes = len(np.unique(y_test))
        cm = confusion_matrix(y_test, y_pred)
        specificity = calculate_specificity(cm)
        sensitivity = calculate_sensitivity(cm)
        # Print the results
        accuracy = accuracy_score(y_test, y_pred)
        fold_acc.append(accuracy)

        # Calculate AUC score
        auc_tot = multiclass_roc_auc_score(y_test, y_pred)
        fold_auc_tot.append(auc_tot)

        auc_class = class_vs_rest_roc_auc_score(y_test, y_pred)
        fold_auc_class.append(auc_class)

        speci.append(specificity)
        sensi.append(sensitivity)

        print('Auc Score', multiclass_roc_auc_score(y_test, y_pred))
        print('Auc Score OvR', class_vs_rest_roc_auc_score(y_test, y_pred))
    # Calculate average scores across folds
    if name == 'rf':
        #acc
        avg_acc = np.mean(fold_acc)
        scores_rf['acc'].append(avg_acc)
        scores_rf['acc'] = scores_rf['acc'][-1]
        ci = stats.t.interval(confidence, len(fold_acc) - 1, loc=np.mean(fold_acc), scale=stats.sem(fold_acc))
        scores_rf['CI_acc'].append(ci)
        scores_rf['SD_acc'].append(np.std(fold_acc))

        #auc_tot
        avg_auc_tot = np.mean(fold_auc_tot)
        scores_rf['auc_tot'].append(avg_auc_tot)
        scores_rf['auc_tot'] = scores_rf['auc_tot'][-1]
        ci = stats.t.interval(confidence, len(fold_auc_tot) - 1, loc=np.mean(fold_auc_tot), scale=stats.sem(fold_auc_tot))
        scores_rf['CI_auc_tot'].append(ci)
        scores_rf['SD_auc_tot'].append(np.std(fold_acc))

        #auc_class
        list_0 = list(d[0] for d in fold_auc_class)
        list_1 = list(d[1] for d in fold_auc_class)
        list_2 = list(d[2] for d in fold_auc_class)

        avg_auc_class = {0: np.mean(list_0), 1: np.mean(list_1), 2: np.mean(list_2)}
        scores_rf['auc_class'].append(avg_auc_class)

        sd_auc_class = {0: np.std(list_0), 1: np.std(list_1), 2: np.std(list_2)}
        scores_rf['SD_auc_class'].append(sd_auc_class)

        ci_0 = stats.t.interval(confidence, len(list_0) - 1, loc=np.mean(list_0), scale=stats.sem(list_0))
        ci_1 = stats.t.interval(confidence, len(list_1) - 1, loc=np.mean(list_1), scale=stats.sem(list_1))
        ci_2 = stats.t.interval(confidence, len(list_2) - 1, loc=np.mean(list_2), scale=stats.sem(list_2))

        CI_auc_class = {0: ci_0, 1: ci_1, 2: ci_2}
        scores_rf['CI_auc_class'].append(CI_auc_class)

        #sensitivity
        list_0 = list(d[0] for d in sensi)
        list_1 = list(d[1] for d in sensi)
        list_2 = list(d[2] for d in sensi)

        avg_sensi_class = {0: np.mean(list_0), 1: np.mean(list_1), 2: np.mean(list_2)}
        scores_rf['Sen_mean'].append(avg_sensi_class)

        sd_sensi_class = {0: np.std(list_0), 1: np.std(list_1), 2: np.std(list_2)}
        scores_rf['Sen_SD'].append(sd_sensi_class)

        ci_0 = stats.t.interval(confidence, len(list_0) - 1, loc=np.mean(list_0), scale=stats.sem(list_0))
        ci_1 = stats.t.interval(confidence, len(list_1) - 1, loc=np.mean(list_1), scale=stats.sem(list_1))
        ci_2 = stats.t.interval(confidence, len(list_2) - 1, loc=np.mean(list_2), scale=stats.sem(list_2))

        CI_sen_class = {0: ci_0, 1: ci_1, 2: ci_2}
        scores_rf['Sen_CI'].append(CI_sen_class)

        #specificity
        list_0 = list(d[0] for d in speci)
        list_1 = list(d[1] for d in speci)
        list_2 = list(d[2] for d in speci)

        avg_speci_class = {0: np.mean(list_0), 1: np.mean(list_1), 2: np.mean(list_2)}
        scores_rf['Spe_mean'].append(avg_speci_class)
        sd_speci_class = {0: np.std(list_0), 1: np.std(list_1), 2: np.std(list_2)}
        scores_rf['Spe_sd'].append(sd_speci_class)

        ci_0 = stats.t.interval(confidence, len(list_0) - 1, loc=np.mean(list_0), scale=stats.sem(list_0))
        ci_1 = stats.t.interval(confidence, len(list_1) - 1, loc=np.mean(list_1), scale=stats.sem(list_1))
        ci_2 = stats.t.interval(confidence, len(list_2) - 1, loc=np.mean(list_2), scale=stats.sem(list_2))

        CI_spe_class = {0: ci_0, 1: ci_1, 2: ci_2}
        scores_rf['Spe_CI'].append(CI_spe_class)

    elif name == 'svc':
        # acc
        avg_acc = np.mean(fold_acc)
        scores_svc['acc'].append(avg_acc)
        scores_svc['acc'] = scores_svc['acc'][-1]
        ci = stats.t.interval(confidence, len(fold_acc) - 1, loc=np.mean(fold_acc), scale=stats.sem(fold_acc))
        scores_svc['CI_acc'].append(ci)
        scores_svc['SD_acc'].append(np.std(fold_acc))

        # auc_tot
        avg_auc_tot = np.mean(fold_auc_tot)
        scores_svc['auc_tot'].append(avg_auc_tot)
        scores_svc['auc_tot'] = scores_svc['auc_tot'][-1]
        ci = stats.t.interval(confidence, len(fold_auc_tot) - 1, loc=np.mean(fold_auc_tot), scale=stats.sem(fold_auc_tot))
        scores_svc['CI_auc_tot'].append(ci)
        scores_svc['SD_auc_tot'].append(np.std(fold_acc))

        # auc_class
        list_0 = list(d[0] for d in fold_auc_class)
        list_1 = list(d[1] for d in fold_auc_class)
        list_2 = list(d[2] for d in fold_auc_class)

        avg_auc_class = {0: np.mean(list_0), 1: np.mean(list_1), 2: np.mean(list_2)}
        scores_svc['auc_class'].append(avg_auc_class)

        sd_auc_class = {0: np.std(list_0), 1: np.std(list_1), 2: np.std(list_2)}
        scores_svc['SD_auc_class'].append(sd_auc_class)

        ci_0 = stats.t.interval(confidence, len(list_0) - 1, loc=np.mean(list_0), scale=stats.sem(list_0))
        ci_1 = stats.t.interval(confidence, len(list_1) - 1, loc=np.mean(list_1), scale=stats.sem(list_1))
        ci_2 = stats.t.interval(confidence, len(list_2) - 1, loc=np.mean(list_2), scale=stats.sem(list_2))

        CI_auc_class = {0: ci_0, 1: ci_1, 2: ci_2}
        scores_svc['CI_auc_class'].append(CI_auc_class)

        # sensitivity
        list_0 = list(d[0] for d in sensi)
        list_1 = list(d[1] for d in sensi)
        list_2 = list(d[2] for d in sensi)

        avg_sensi_class = {0: np.mean(list_0), 1: np.mean(list_1), 2: np.mean(list_2)}
        scores_svc['Sen_mean'].append(avg_sensi_class)

        sd_sensi_class = {0: np.std(list_0), 1: np.std(list_1), 2: np.std(list_2)}
        scores_svc['Sen_SD'].append(sd_sensi_class)

        ci_0 = stats.t.interval(confidence, len(list_0) - 1, loc=np.mean(list_0), scale=stats.sem(list_0))
        ci_1 = stats.t.interval(confidence, len(list_1) - 1, loc=np.mean(list_1), scale=stats.sem(list_1))
        ci_2 = stats.t.interval(confidence, len(list_2) - 1, loc=np.mean(list_2), scale=stats.sem(list_2))

        CI_sen_class = {0: ci_0, 1: ci_1, 2: ci_2}
        scores_svc['Sen_CI'].append(CI_sen_class)

        # specificity
        list_0 = list(d[0] for d in speci)
        list_1 = list(d[1] for d in speci)
        list_2 = list(d[2] for d in speci)

        avg_speci_class = {0: np.mean(list_0), 1: np.mean(list_1), 2: np.mean(list_2)}
        scores_svc['Spe_mean'].append(avg_speci_class)

        sd_speci_class = {0: np.std(list_0), 1: np.std(list_1), 2: np.std(list_2)}
        scores_svc['Spe_sd'].append(sd_speci_class)

        ci_0 = stats.t.interval(confidence, len(list_0) - 1, loc=np.mean(list_0), scale=stats.sem(list_0))
        ci_1 = stats.t.interval(confidence, len(list_1) - 1, loc=np.mean(list_1), scale=stats.sem(list_1))
        ci_2 = stats.t.interval(confidence, len(list_2) - 1, loc=np.mean(list_2), scale=stats.sem(list_2))

        CI_spe_class = {0: ci_0, 1: ci_1, 2: ci_2}
        scores_svc['Spe_CI'].append(CI_spe_class)

Pipeline(steps=[('smoteen', SMOTEENN(random_state=0)),
                ('clf',
                 RandomForestClassifier(class_weight='balanced',
                                        n_estimators=3500, random_state=0))])
Auc Score 0.7149577246643685
Auc Score OvR {0: 0.7478354978354979, 1: 0.6798245614035089, 2: 0.7172131147540984}
Pipeline(steps=[('smoteen', SMOTEENN(random_state=0)),
                ('clf',
                 RandomForestClassifier(class_weight='balanced',
                                        n_estimators=3500, random_state=0))])
Auc Score 0.6679537268406984
Auc Score OvR {0: 0.5562770562770563, 1: 0.6798245614035089, 2: 0.76775956284153}
Pipeline(steps=[('smoteen', SMOTEENN(random_state=0)),
                ('clf',
                 RandomForestClassifier(class_weight='balanced',
                                        n_estimators=3500, random_state=0))])
Auc Score 0.6051818726347028
Auc Score OvR {0: 0.7326839826839826, 1: 0.6245283018867924, 2: 0.458333333333333

In [117]:

for name, score in scores_svc.items():
    print('svc', f'{name}: {score}')

for name, score in scores_rf.items():
    print('rf', f'{name}: {score}')

svc acc: 0.4969558599695586
svc SD_acc: [0.1012634222072523]
svc CI_acc: [(0.4622072284767971, 0.53170449146232)]
svc auc_tot: 0.551275614928692
svc SD_auc_tot: [0.1012634222072523]
svc CI_auc_tot: [(0.5311731135508174, 0.5713781163065667)]
svc auc_class: [{0: 0.578836517269353, 1: 0.4851501168773319, 2: 0.589840210639391}]
svc SD_auc_class: [{0: 0.09709967168877058, 1: 0.06803595529519324, 2: 0.0875823113647351}]
svc CI_auc_class: [{0: (0.5455166803920688, 0.6121563541466373), 1: (0.4618035195840662, 0.5084967141705976), 2: (0.5597862642433397, 0.6198941570354423)}]
svc Sen_mean: [{0: 33.3994708994709, 1: 53.86481869710381, 2: 41.114672364672366}]
svc Sen_CI: [{0: (26.057629386279714, 40.74131241266208), 1: (47.55647364592472, 60.173163748282896), 2: (30.90591197971979, 51.32343274962494)}]
svc Sen_SD: [{0: 21.395374867752196, 1: 18.3835904006651, 2: 29.7500640648099}]
svc Spe_mean: [{0: 82.36783255439973, 1: 43.16520467836257, 2: 76.85336976320585}]
svc Spe_CI: [{0: (79.0064440063375